# Analiza - sprawdzenie bardzo prostej strategii opartej jedynie na dwóch formacjach świecowych + "zasada 11/20" + dodatkowo RSI

<h3>
<center>
Robert Nowotniak <rnowotniak@gmail.com><br/>
w oparciu o metodykę Fraktal Trader Moduł 1, formacje świecowe
</center>
</h3>

* [Eksperyment 1) Analiza młotków - EURUSD 1h](#eksperyment1)

* [Eksperyment 2) Spadające gwiazdy - Dow Jones 1h](#eksperyment2)

* [Eksperyment 3) Analiza zbiorcza dla wszystkich walorów - Młotki](#eksperyment3)

* [Eksperyment 3) - Podsumowanie zbiorcze](#eksperyment3summary)
 
* [Eksperyment 4) Analiza zbiorcza dla wszystkich walorów - Spadające gwiazdy](#eksperyment4)

* [Eksperyment 4) - Podsumowanie zbiorcze](#eksperyment4summary)

In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import talib
import talib.abstract as ta
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import seaborn as sns
import glob, os.path
sns.set_style('whitegrid')
%matplotlib inline

In [2]:
def ohlc(r):
    return r['open'], r['high'], r['low'], r['close']

## Wczytanie specyfikacji wszystkich instrumentów

Specyfikacja instrumentów wygenerowana w MetaTraderze skryptem MQL allmarketdata.ex4

In [3]:
specs = pd.read_csv('instruments.csv', sep=';', index_col=0)

# XXX - nie da się tego wyciągnąć bezpośrednio ze specyfikacji ??
def setPipSize(x):
    if x['Pr. Mode'] == 'Forex':
        if x.name.lower().endswith('jpy'):
            return 0.01
        return 0.0001
    return float(x['Tick Size']) # XXX - uproszczenie, nie jestem pewien, czy to jest prawidłowe
    if x['Symbol'].endswith('JPY'):
            return 0.01
        # elif x['Symbol'].endswith('HUF'):  # XXX Hungarian Forint ???
        #return 0.0001
    return 1

specs['Pip'] = specs.apply(setPipSize, axis=1)

### Wczytanie historycznych danych wszystkich walorów do klasy Instrument

In [4]:
class Instrument:
    def __init__(self, name):
        self.name = name
        self.spec = None
        self.results = None
        self.balance = 0  # +/- gain value
        self.stats = {} # PROFIT: ...,  LOSS: ...
        self.success_rate = None
        self.data = pd.read_csv('data/%s60.csv' % name, index_col = False,
                   names=['day', 'hour', 'open', 'high', 'low', 'close', 'volume'],
                               dtype={'open':np.float64,'high':np.float64,
                                     'low':np.float64,'close':np.float64,'volume':np.float64}
                               )
        self.data['date'] = pd.DatetimeIndex(self.data.day + ' ' + self.data.hour)
        del self.data['day']
        del self.data['hour']
        self.data.set_index('date', inplace = True)
        
        # Calculate RSI14 for this instrument
        self.rsi = ta.RSI(self.data, 14)
    
    def __str__(self):
        s = self.name
        if self.data is not None:
            s = s + ' ' + str(self.data.shape)
        return s

INSTRUMENTS = {}        
        
for f in glob.glob('data/??????60.csv'):
    name = os.path.basename(f).replace('60.csv', '')
    print(name)
    inst = Instrument(name)
    inst.spec = specs.ix[name]
    INSTRUMENTS[name] = inst
 

USDCHF
EURDKK
EURUSD
EURPLN
EURNOK
EURCHF
GBPCAD
USDJPY
GBPUSD
CADJPY
EURRUB
SILVER
AUDCAD
EURSEK
AUDJPY
AUDCHF
EURAUD
USDCAD
EURHKD
AUDNZD


In [5]:
INSTRUMENTS['EURRUB'].rsi['2017/06/16 10:00']

67.19948864159953

In [6]:
INSTRUMENTS['EURNOK'].spec

Description      Euro vs Norwegian Krone
Digits                                 5
Spread                               614
Swap Long                         -85.69
Swap Short                        -30.59
Swap Type                         points
Stop Level                           620
Lot Size                          100000
Lot Min                             0.01
Lot Step                            0.01
Tick Value                      0.443566
Tick Size                          1e-05
Trade Allowed                        yes
Pr. Mode                           Forex
Pip                               0.0001
Name: EURNOK, dtype: object

Delete all non-currencies:

In [7]:
todelete = []
for iname in INSTRUMENTS:
    if INSTRUMENTS[iname].spec['Pr. Mode'] != 'Forex':
        todelete.append(iname)
for iname in todelete:
    del INSTRUMENTS[iname]
for iname in INSTRUMENTS:
    print(iname)

CADJPY
AUDJPY
EURNOK
EURAUD
USDJPY
AUDCHF
AUDCAD
EURCHF
EURSEK
AUDNZD
GBPCAD
USDCAD
GBPUSD
EURUSD
USDCHF
EURPLN


## Zasada 11 / 20 - porównywanie rozmiaru świecy z 20 poprzednimi

In [8]:
def zasada1120(inst, t):
    
    BACKSIZE = 20         # XXX Wyciągnąć gdzieś do konfiguracji
    CANDLES_REQUIRED = 11 # XXX
    
    t = pd.Timestamp(t)
    i = inst.data.index.get_loc(t, 'nearest')
    o, h, l, c = ohlc(inst.data.iloc[i])
    size = h - l
   
    j = 1
    smaller_candles = 0
    while j <= BACKSIZE and i - j >= 0 and smaller_candles < CANDLES_REQUIRED:
        o, h, l, c = ohlc(inst.data.iloc[i - j])
        if size > h - l:
            smaller_candles += 1
        j += 1
    if smaller_candles >= CANDLES_REQUIRED:
        return True
    
    return False
    
#zasada1120(INSTRUMENTS['US30Cash'], '2017/06/20 22:00')

## Sprawdzanie, czy dany sygnał był prawidłowy(czy zarobił) -- B. WAŻNY FRAGMENT
### (ta funkcja docelowo byłaby jednym z głównych elementów pełnego symulatora strategii)

In [21]:
# returns:
#   (PROFIT/LOSS, gain value, nswaps, sp, tp, size)
def sprawdz(inst, t, buysell = 'buy'):
    t = pd.Timestamp(t)
    i = inst.data.index.get_loc(t, 'nearest')
    o, h, l, c = ohlc(inst.data.iloc[i])
    c0 = c
    size = h - l
    
    spread = 3 # XXX  US30
    spread = 0.0003 # XXX EURUSD
    spread = float(inst.spec['Spread']) / 10**int(inst.spec['Digits'])  # XXX
    #print(spread)
    PROFIT_TO_RISK = 3  # XXX Wyciagnac do konfiguracji
    
    if buysell == 'buy':
        sl = l - inst.spec['Pip'] # 1 punkt pod formacja
        risk = c - sl + spread # -> mozliwa strata = roznica poziomow + spread
        tp = c + spread + risk * PROFIT_TO_RISK # -> mozliwy zysk = roznica poziomow - spread = risk*PTR
    elif buysell == 'sell':
        sl = h + inst.spec['Pip'] + spread # 1 punkt nad formacja + spread (chyba z zalozenia ze spada szybciej)
        risk = sl - c + spread # mozliwa strata = roznica poziomow + spread
        tp = c - risk * PROFIT_TO_RISK - spread  # -> mozliwy zysk = roznica poziomow - spread = risk*PTR
    else:
        raise Exception('buysell')
    # print(sl, tp)
    
    before_midnght = inst.data.index[i].hour > 12
    nswaps = 0
    while i < inst.data.shape[0] - 1:
        i += 1
        o, h, l, c = ohlc(inst.data.iloc[i])
        
        if before_midnght and inst.data.index[i].hour < 12:
            # print('SWAP')
            nswaps += 1
        before_midnght = inst.data.index[i].hour > 12
        
        result = None
        if (buysell == 'buy' and h >= tp) or (buysell == 'sell' and l <= tp):
            # print(inst.data.index[i])
            result = 'PROFIT'
            
        # Sprawdzenie warunku 10pips
        how_many_pips = 8.
        if h - c0 >= how_many_pips * inst.spec['Pip'] and buysell == 'buy':
            result = 'PROFIT'
            return result, how_many_pips * inst.spec['Pip'] - spread, nswaps, sl, tp, size 
        if c0 - l >= how_many_pips * inst.spec['Pip'] and buysell == 'sell':
            result = 'PROFIT'
            return result, how_many_pips * inst.spec['Pip'] - spread, nswaps, sl, tp, size     
        #################################
            
        if (buysell == 'buy' and l <= sl) or (buysell == 'sell' and h >= sl):
            # print(inst.data.index[i])
            if result == 'PROFIT':
                return 'Unknown'
            result = 'LOSS'
        if result == 'PROFIT':
            return result, PROFIT_TO_RISK * risk, nswaps, sl, tp, size #, zasada1120(inst, t)
        if result == 'LOSS':
            return result, -risk, nswaps, sl, tp, size #, zasada1120(inst, t)

t = '2017/06/22 01:00'
print(sprawdz(INSTRUMENTS['EURUSD'], t))

('LOSS', -0.00042000000000001802, 0, 1.11635, 1.1180300000000001, 0.00029000000000012349)


## Definicja wybranych formacji świecowych - funkcje wykrywające

In [22]:
def jestMlotek(row):
    o, h, l, c = ohlc(row)
    if c > o and c - o <= 0.4 * (h - l):
        if h - c <= 0.14 * (h - l):
            return True
    if c < o and o - c <= 0.4 * (h - l):
        if h - o <= 0.14 * (h - l):
            return True    
    return False

def jestSpadajacaGwiazda(row):
    o, h, l, c = ohlc(row)
    if c > o and c - o <= 0.4 * (h - l):
        if o - l <= 0.14 * (h - l):
            return True
    if c < o and o - c <= 0.4 * (h - l):
        if c - l <= 0.14 * (h - l):
            return True    
    return False

# ...
# Tutaj dalej docelowo powinny być zaimplementowane inne formacje (przenikania, zasłona ciemnej chmury itp)
# ...

<a id="eksperyment1"></a>
## Eksperyment 1) Analiza młotków - EURUSD 1h

In [23]:
inst = INSTRUMENTS['EURUSD']

stats = {}
balance = 0.
for idx, row in inst.data['2017/01/01':'2017/07/01'].iterrows():
    if jestMlotek(row) and zasada1120(inst,idx) and inst.rsi[idx] <= 30:
        res = sprawdz(inst, idx)
        if not res:
            print(idx, ' ', res)
            continue        
        print(idx, ' ', '%6s'%res[0], ' ', end='')
        for _ in res[1:]:
            print('%.5f'%_, ' ', end='')
        print()
        if type(res) == tuple:
            balance += res[1] # XXX tu ewentualnie uwzgledniac swap * nswaps
            if res[0] not in stats:
                stats[res[0]] = 0
            stats[res[0]] += 1
print(balance)
print(stats)
if 'PROFIT' in stats:
    print("Success rate: %.2f%%" % (100.0*stats['PROFIT']/ sum(stats.values())))

2017-02-07 10:00:00     LOSS  -0.00154  0.00000  1.06597  1.07213  0.00226  
2017-02-10 16:00:00   PROFIT  0.00062  0.00000  1.06060  1.06748  0.00167  
2017-02-22 12:00:00   PROFIT  0.00062  0.00000  1.04924  1.05516  0.00164  
2017-06-15 19:00:00   PROFIT  0.00062  0.00000  1.11375  1.11851  0.00121  
0.00032
{'LOSS': 1, 'PROFIT': 3}
Success rate: 75.00%


<a id="eksperyment2"></a>
## Eksperyment 2) Spadające gwiazdy - EURUSD 1h

In [24]:
inst = INSTRUMENTS['EURUSD']

balance = 0.
stats = {}
for idx, row in inst.data['2017/01/01':'2017/07/01'].iterrows():
    if jestSpadajacaGwiazda(row) and zasada1120(inst,idx) and inst.rsi[idx] >= 70:
        res = sprawdz(inst, idx, 'sell') # XXX <- SELL
        if not res:
            print(idx, ' ', res)
            continue
        print(idx, ' ', '%6s'%res[0], ' ', end='')
        for _ in res[1:]:
            print('%.5f'%_, ' ', end='')
        print()
        if type(res) == tuple:
            balance += res[1] # XXX tu ewentualnie uwzgledniac swap * nswaps
            if res[0] not in stats:
                stats[res[0]] = 0
            stats[res[0]] += 1
print(balance)  
print(stats)
if 'PROFIT' in stats:
    print("Success rate: %.2f%%" % (100.0*stats['PROFIT']/ sum(stats.values())))

2017-03-09 16:00:00   PROFIT  0.00062  0.00000  1.06171  1.05131  0.00354  
2017-04-12 23:00:00   PROFIT  0.00062  1.00000  1.06772  1.06132  0.00175  
2017-04-13 00:00:00   PROFIT  0.00062  0.00000  1.06751  1.06367  0.00108  
2017-06-14 17:00:00   PROFIT  0.00062  0.00000  1.12979  1.12139  0.00257  
0.00248
{'PROFIT': 4}
Success rate: 100.00%


<a id="eksperyment3"></a>
## Eksperyment 3) Analiza zbiorcza dla wszystkich walorów - Młotki

In [25]:
%%time

for name in INSTRUMENTS:
    print(name)
    inst = INSTRUMENTS[name]
    inst.results = pd.DataFrame( columns = range(7) )

    stats = {}
    balance = 0.
    for idx, row in inst.data['2017/01/01':].iterrows():  # XXX Month
        if jestMlotek(row) and zasada1120(inst,idx): #  and inst.rsi[idx] <= 30: # XXX jestMlotek
            res = sprawdz(inst, idx) # XXX buy/sell
            if not res or type(res) == str:
                continue        
            inst.results = inst.results.append([ (idx,) + res ])
            if type(res) == tuple:
                balance += res[1] # XXX tu ewentualnie uwzgledniac swap * nswaps
                if res[0] not in stats:
                    stats[res[0]] = 0
                stats[res[0]] += 1
    inst.results.columns = ['date', 'PL', 'gv', 'nswaps', 'sl', 'tp', 'size']
    inst.results.set_index('date', inplace=True)
    inst.balance = balance
    inst.stats = stats
    inst.success_rate = 0
    if 'PROFIT' in stats:
        inst.success_rate = stats['PROFIT']/ sum(stats.values())

CADJPY
AUDJPY
EURNOK
EURAUD
USDJPY
AUDCHF
AUDCAD
EURCHF
EURSEK
AUDNZD
GBPCAD
USDCAD
GBPUSD
EURUSD
USDCHF
EURPLN
CPU times: user 22.7 s, sys: 43.5 ms, total: 22.7 s
Wall time: 22.7 s


In [14]:
for iname in sorted(INSTRUMENTS.keys()):
    inst = INSTRUMENTS[iname]
    if inst.results is not None:
        display(HTML('<h3>%s</h3>' % iname))
        display(HTML('<b>Balance: </b> %f &nbsp;&nbsp;&nbsp;&nbsp; <b>Trades:</b>%s' % (inst.balance, inst.stats)))
        display(HTML('<b>Success rate: </b> %.2f%%' % (100. * inst.success_rate)))
        print("One month details:")
        #display(inst.results['2017/06/01':])        
        display(inst.results[:])        

One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-03-01 09:00:00,PROFIT,0.00068,0.0,1.01758,1.03226,0.00340
2017-03-09 00:00:00,LOSS,-0.00132,0.0,1.01401,1.01929,0.00130
2017-03-10 13:00:00,PROFIT,0.00068,0.0,1.01386,1.01966,0.00129
2017-03-15 23:00:00,LOSS,-0.00252,1.0,1.02252,1.03260,0.00298
2017-03-16 01:00:00,PROFIT,0.00068,0.0,1.02316,1.02940,0.00120
2017-03-16 14:00:00,LOSS,-0.00185,0.0,1.02211,1.02951,0.00163
2017-03-21 05:00:00,PROFIT,0.00068,0.0,1.02794,1.03506,0.00143
2017-03-29 00:00:00,PROFIT,0.00068,0.0,1.02000,1.02644,0.00127
2017-04-03 10:00:00,PROFIT,0.00068,0.0,1.01219,1.01863,0.00138


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-03-01 16:00:00,LOSS,-0.00118,0.0,0.77332,0.77804,0.00144
2017-03-08 11:00:00,LOSS,-0.00163,0.0,0.76516,0.77168,0.00190
2017-03-08 16:00:00,LOSS,-0.00151,0.0,0.76386,0.76990,0.00121
2017-03-09 00:00:00,LOSS,-0.00129,0.0,0.76241,0.76757,0.00115
2017-03-10 10:00:00,PROFIT,0.00064,0.0,0.75983,0.76675,0.00166
2017-03-13 08:00:00,PROFIT,0.00064,0.0,0.76169,0.77053,0.00184
2017-03-16 01:00:00,LOSS,-0.00133,0.0,0.76947,0.77479,0.00113
2017-03-16 13:00:00,LOSS,-0.00168,0.0,0.76563,0.77235,0.00135
2017-03-21 20:00:00,LOSS,-0.00189,0.0,0.76449,0.77205,0.00165


One month details:


...

,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-02-27 16:00:00,PROFIT,0.00079,0.0,1.30814,1.31810,0.00271
2017-03-03 21:00:00,LOSS,-0.00209,0.0,1.33817,1.34653,0.00218
2017-03-07 08:00:00,PROFIT,0.00079,0.0,1.33829,1.34393,0.00125
2017-03-08 03:00:00,PROFIT,0.00079,0.0,1.33956,1.34600,0.00148
2017-03-09 08:00:00,PROFIT,0.00079,0.0,1.34843,1.35211,0.00126
2017-03-10 13:00:00,PROFIT,0.00079,0.0,1.34880,1.35412,0.00114
2017-03-10 19:00:00,LOSS,-0.00167,0.0,1.34519,1.35187,0.00164
2017-03-10 21:00:00,LOSS,-0.00148,0.0,1.34514,1.35106,0.00153
2017-03-10 22:00:00,PROFIT,0.00079,0.0,1.34450,1.35178,0.00181


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-02-28 15:00:00,LOSS,-0.00183,0.0,1.00374,1.01106,0.00184
2017-03-09 08:00:00,LOSS,-0.00094,0.0,1.01454,1.01830,0.00099
2017-03-10 14:00:00,PROFIT,0.00080,0.0,1.01155,1.01731,0.00127
2017-03-13 17:00:00,PROFIT,0.00080,1.0,1.00588,1.01036,0.00142
2017-03-16 02:00:00,PROFIT,0.00080,0.0,0.99825,1.00317,0.00167
2017-03-16 08:00:00,LOSS,-0.00120,0.0,0.99801,1.00281,0.00142
2017-03-16 09:00:00,LOSS,-0.00221,0.0,0.99769,1.00653,0.00215
2017-03-20 19:00:00,PROFIT,0.00080,1.0,0.99741,1.00345,0.00140
2017-03-21 03:00:00,PROFIT,0.00080,0.0,0.99652,1.00212,0.00114


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-01-02 18:00:00,PROFIT,0.084,0.0,117.090,118.282,0.288
2017-01-02 20:00:00,PROFIT,0.084,0.0,117.171,118.371,0.287
2017-01-03 22:00:00,PROFIT,0.084,1.0,117.501,118.469,0.224
2017-01-04 11:00:00,LOSS,-0.193,0.0,117.506,118.278,0.258
2017-01-04 13:00:00,PROFIT,0.084,0.0,117.345,118.309,0.245
2017-01-05 07:00:00,LOSS,-0.428,0.0,116.062,117.774,0.458
2017-01-05 12:00:00,LOSS,-0.350,0.0,116.412,117.812,0.366
2017-01-06 01:00:00,PROFIT,0.084,0.0,115.055,116.247,0.416
2017-01-09 11:00:00,PROFIT,0.084,0.0,116.849,117.777,0.312


In [15]:
INSTRUMENTS['USDCHF'].results['gv'].sum()

-0.010070000000000048

<a id="eksperyment3summary"></a>
### Podsumowanie zbiorcze - Młotki

In [16]:
posivite_instruments = 0
for iname in sorted(INSTRUMENTS.keys()):
    print('%11s   %g' % (iname, INSTRUMENTS[iname].balance))
    if INSTRUMENTS[iname].balance > 0:
        posivite_instruments += 1
print()
print("%d/%d (%.2f%%)" % (posivite_instruments, len(INSTRUMENTS),  100. * posivite_instruments / len(INSTRUMENTS)))
print()

total_nloss = 0
total_nprofit = 0
for iname in sorted(INSTRUMENTS.keys()):
    total_nloss += INSTRUMENTS[iname].stats.get('LOSS', 0)
    total_nprofit += INSTRUMENTS[iname].stats.get('PROFIT', 0)
    #print(iname, str(INSTRUMENTS[iname].stats))
    #if INSTRUMENTS[iname].balance > 0:
    #    posivite_instruments += 1
print("%d %d" % (total_nprofit, total_nprofit + total_nloss))

     AUDCAD   -0.02265
     AUDCHF   -0.03736
     AUDJPY   -2.778
     AUDNZD   -0.02535
     CADJPY   -0.885
     EURAUD   -0.02359
     EURCHF   -0.02897
     EURNOK   -0.46468
     EURPLN   -0.56895
     EURSEK   -0.23347
     EURUSD   -0.00987
     GBPCAD   -0.0207
     GBPUSD   -0.01637
     USDCAD   0.00265
     USDCHF   -0.01007
     USDJPY   -0.923

1/16 (6.25%)

632 975


<a id='eksperyment4'></a>
## Eksperyment 4) Analiza zbiorcza dla wszystkich walorów - Spadające gwiazdy

In [17]:
%%time

for iname in sorted(INSTRUMENTS.keys()):
    print(iname)
    inst = INSTRUMENTS[iname]
    inst.results = pd.DataFrame( columns = range(7) )

    stats = {}
    balance = 0.
    for idx, row in inst.data['2017/01/01':].iterrows():  # XXX Month
        if jestSpadajacaGwiazda(row) and zasada1120(inst,idx): # and inst.rsi[idx] >= 70: # XXX jestMlotek
            res = sprawdz(inst, idx, 'sell') # XXX buy/sell
            if not res or type(res) == str:
                continue        
            inst.results = inst.results.append([ (idx,) + res ])
            if type(res) == tuple:
                balance += res[1] # XXX tu ewentualnie uwzgledniac swap * nswaps
                if res[0] not in stats:
                    stats[res[0]] = 0
                stats[res[0]] += 1
    inst.results.columns = ['date', 'PL', 'gv', 'nswaps', 'sl', 'tp', 'size']
    inst.results.set_index('date', inplace=True)
    inst.balance = balance
    inst.stats = stats
    inst.success_rate = 0
    if 'PROFIT' in stats:
        inst.success_rate = stats['PROFIT']/ sum(stats.values())

AUDCAD
AUDCHF
AUDJPY
AUDNZD
CADJPY
EURAUD
EURCHF
EURNOK
EURPLN
EURSEK
EURUSD
GBPCAD
GBPUSD
USDCAD
USDCHF
USDJPY
CPU times: user 22.4 s, sys: 16.6 ms, total: 22.4 s
Wall time: 22.3 s


In [18]:
for iname in sorted(INSTRUMENTS.keys()):
    inst = INSTRUMENTS[iname]
    if inst.results is not None:
        display(HTML('<h3>%s</h3>' % iname))
        display(HTML('<b>Balance: </b> %f &nbsp;&nbsp;&nbsp;&nbsp; <b>Trades:</b>%s' % (inst.balance, inst.stats)))
        display(HTML('<b>Success rate: </b> %.2f%%' % (100. * inst.success_rate)))
        print("One month details:")
        #display(inst.results['2017/06/01':])        
        display(inst.results[:])        

One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-02-27 06:00:00,PROFIT,0.00068,0.0,1.00969,1.00045,0.00220
2017-03-01 04:00:00,PROFIT,0.00068,0.0,1.02324,1.01000,0.00306
2017-03-03 13:00:00,LOSS,-0.00190,0.0,1.01443,1.00683,0.00130
2017-03-06 15:00:00,PROFIT,0.00068,0.0,1.01897,1.01185,0.00162
2017-03-12 23:00:00,LOSS,-0.00181,1.0,1.01645,1.00921,0.00162
2017-03-13 09:00:00,PROFIT,0.00068,0.0,1.02069,1.01261,0.00146
2017-03-15 19:00:00,LOSS,-0.00247,0.0,1.02332,1.01344,0.00193
2017-03-16 18:00:00,LOSS,-0.00182,0.0,1.02374,1.01646,0.00195
2017-03-17 17:00:00,PROFIT,0.00068,0.0,1.02974,1.02002,0.00232


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-02-27 06:00:00,PROFIT,0.00064,0.0,0.77669,0.76877,0.00169
2017-02-27 10:00:00,PROFIT,0.00064,0.0,0.77566,0.76738,0.00125
2017-02-28 10:00:00,PROFIT,0.00064,0.0,0.77451,0.76583,0.00149
2017-03-01 04:00:00,PROFIT,0.00064,0.0,0.77452,0.76540,0.00249
2017-03-02 04:00:00,PROFIT,0.00064,0.0,0.77496,0.76672,0.00131
2017-03-07 07:00:00,PROFIT,0.00064,0.0,0.77181,0.76489,0.00098
2017-03-10 15:00:00,PROFIT,0.00064,0.0,0.76450,0.75506,0.00279
2017-03-14 13:00:00,LOSS,-0.00206,0.0,0.76347,0.75523,0.00125
2017-03-15 19:00:00,LOSS,-0.00223,0.0,0.76733,0.75841,0.00170


One month details:


...

,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-02-28 09:00:00,LOSS,-0.00177,0.0,1.31864,1.31156,0.00134
2017-02-28 20:00:00,LOSS,-0.00208,0.0,1.32892,1.32060,0.00157
2017-03-01 15:00:00,PROFIT,0.00079,0.0,1.33480,1.32516,0.00268
2017-03-02 17:00:00,LOSS,-0.00259,0.0,1.33958,1.32922,0.00217
2017-03-03 18:00:00,PROFIT,0.00079,0.0,1.34371,1.33543,0.00249
2017-03-07 15:00:00,LOSS,-0.00205,0.0,1.34230,1.33410,0.00165
2017-03-08 10:00:00,LOSS,-0.00138,0.0,1.34353,1.33801,0.00128
2017-03-08 11:00:00,LOSS,-0.00207,0.0,1.34479,1.33651,0.00217
2017-03-13 20:00:00,LOSS,-0.00171,1.0,1.34558,1.33874,0.00153


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-02-28 10:00:00,PROFIT,0.00080,0.0,1.00876,1.00040,0.00183
2017-03-01 03:00:00,PROFIT,0.00080,0.0,1.00860,1.00056,0.00166
2017-03-01 16:00:00,PROFIT,0.00080,0.0,1.01314,1.00546,0.00161
2017-03-07 18:00:00,LOSS,-0.00170,1.0,1.01452,1.00772,0.00136
2017-03-09 21:00:00,LOSS,-0.00116,0.0,1.01085,1.00621,0.00137
2017-03-15 18:00:00,LOSS,-0.00155,0.0,1.00881,1.00261,0.00117
2017-03-15 23:00:00,PROFIT,0.00080,1.0,1.00170,0.99562,0.00139
2017-03-20 10:00:00,PROFIT,0.00080,0.0,0.99876,0.99236,0.00124
2017-03-21 01:00:00,PROFIT,0.00080,0.0,1.00047,0.99291,0.00155


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-01-02 23:00:00,PROFIT,0.084,1.0,117.677,116.817,0.201
2017-01-03 17:00:00,PROFIT,0.084,0.0,118.628,116.164,0.635
2017-01-04 14:00:00,PROFIT,0.084,0.0,117.730,116.786,0.225
2017-01-11 17:00:00,PROFIT,0.084,0.0,116.878,115.590,0.284
2017-01-17 09:00:00,PROFIT,0.084,0.0,113.595,112.295,0.323
2017-01-17 12:00:00,PROFIT,0.084,0.0,113.319,112.235,0.270
2017-01-19 19:00:00,PROFIT,0.084,0.0,115.642,114.626,0.345
2017-01-26 23:00:00,PROFIT,0.084,1.0,114.745,113.721,0.287
2017-01-30 14:00:00,PROFIT,0.084,0.0,114.802,113.958,0.191


<a id="eksperyment4summary"></a>
### Podsumowanie zbiorcze - spadające gwiazdy

In [19]:
posivite_instruments = 0
for iname in sorted(INSTRUMENTS.keys()):
    print('%11s   %g' % (iname, INSTRUMENTS[iname].balance))
    if INSTRUMENTS[iname].balance > 0:
        posivite_instruments += 1
print()
print("%d/%d (%.2f%%)" % (posivite_instruments, len(INSTRUMENTS),  100. * posivite_instruments / len(INSTRUMENTS)))
print()

total_nloss = 0
total_nprofit = 0
for iname in sorted(INSTRUMENTS.keys()):
    total_nloss += INSTRUMENTS[iname].stats.get('LOSS', 0)
    total_nprofit += INSTRUMENTS[iname].stats.get('PROFIT', 0)
    #print(iname, str(INSTRUMENTS[iname].stats))
    #if INSTRUMENTS[iname].balance > 0:
    #    posivite_instruments += 1
print("%d %d" % (total_nprofit, total_nprofit + total_nloss))

     AUDCAD   -0.03371
     AUDCHF   -0.02367
     AUDJPY   -1.99
     AUDNZD   -0.03827
     CADJPY   -3.352
     EURAUD   -0.03382
     EURCHF   -0.01501
     EURNOK   -0.43
     EURPLN   -0.22746
     EURSEK   -0.36806
     EURUSD   0.00704
     GBPCAD   -0.03759
     GBPUSD   -0.01182
     USDCAD   -0.02812
     USDCHF   -0.0124
     USDJPY   -2.347

1/16 (6.25%)

599 901


In [20]:
INSTRUMENTS['AUDCAD'].spec

Description      AUSTRALIAN DOLLAR VS CANADIAN DOLLAR
Digits                                              5
Spread                                             32
Swap Long                                        -0.9
Swap Short                                      -8.19
Swap Type                                      points
Stop Level                                        160
Lot Size                                       100000
Lot Min                                          0.01
Lot Step                                         0.01
Tick Value                                    2.83951
Tick Size                                       1e-05
Trade Allowed                                     yes
Pr. Mode                                        Forex
Pip                                            0.0001
Name: AUDCAD, dtype: object